In [21]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm
import atlite

The objective of this code is to use the PV model we selected to obtain a long-term PV CF time series that we can store and later use to run our analysis. The process is the same we did for PV but this time we consider the locations of farms projected for 2030.

## Necessary steps:

Step 1: Load the ERA5 data and create a cutout

Step 2: Load the farm data

Step 3: Run the atlite PV model for the entire period

Step 3: Save the data into a file to use later in the analysis

### Step 1: Load the ERA5 data and create a cutout

Load the ERA5 data

In [ ]:
ds_temperature_2018_2023 = xr.open_dataset('../data/ERA5/ERA5_2m_temperature_soil_temperature_level_4_hourly_2018_2023.nc')
ds_radiation_sfc_2018_2023 = xr.open_dataset('../data/ERA5/ERA5_surface_net_solar_radiation_surface_solar_radiation_downwards_hourly_2018_2023.nc')
ds_radiation_toa_2018_2023 = xr.open_dataset('../data/ERA5/ERA5_toa_incident_solar_radiation_total_sky_direct_solar_radiation_at_surface_hourly_2018_2023.nc')

ds_temperature_2018_2023 = ds_temperature_2018_2023.reduce(np.nanmean, dim='expver',keep_attrs=True)
ds_radiation_sfc_2018_2023 = ds_radiation_sfc_2018_2023.reduce(np.nanmean, dim='expver',keep_attrs=True)
ds_radiation_toa_2018_2023 = ds_radiation_toa_2018_2023.reduce(np.nanmean, dim='expver',keep_attrs=True)

In [ ]:
ds_temperature_1988_2017 = xr.open_mfdataset(['../data/ERA5/ERA5_2m_temperature_soil_temperature_level_4_hourly_{}_{}.nc'.format(yy, yy+5) for yy in np.arange(1988, 2013, 6, dtype=int)])
ds_radiation_sfc_1988_2017 = xr.open_mfdataset(['../data/ERA5/ERA5_surface_net_solar_radiation_surface_solar_radiation_downwards_hourly_{}_{}.nc'.format(yy, yy+5) for yy in np.arange(1988, 2013, 6, dtype=int)])
ds_radiation_toa_1988_2017 = xr.open_mfdataset(['../data/ERA5/ERA5_toa_incident_solar_radiation_total_sky_direct_solar_radiation_at_surface_hourly_{}_{}.nc'.format(yy, yy+5) for yy in np.arange(1988, 2013, 6, dtype=int)])

In [ ]:
list_files_era5_old = []
filepath_base = '../data/ERA5/ERA5_all_variables_{}_{:02d}.nc'
for year in np.arange(1940,1988, dtype=int):
    for month in np.arange(1,13, dtype=int):
        list_files_era5_old.append(filepath_base.format(year, month))

In [25]:
ds_1940_1987 = xr.open_mfdataset(list_files_era5_old)

In [26]:
ds_1940_1987 = ds_1940_1987[['tisr', 'fdir', 'ssr', 'ssrd', 't2m', 'stl4']]

In [27]:
ds = xr.concat([ds_1940_1987,
                xr.merge([ds_temperature_1988_2017, ds_radiation_sfc_1988_2017, ds_radiation_toa_1988_2017]),
                xr.merge([ds_temperature_2018_2023, ds_radiation_sfc_2018_2023, ds_radiation_toa_2018_2023])
                ], dim='time')

In [28]:
cutout_ie = atlite.cutout.get_cutout_from_era5_data('path', ds, ['influx', 'temperature'])

### Step 2: Load the farm data

In [ ]:
path_farms = '../data/EirGrid/2030_planned_capacity.csv'
df_farms = pd.read_csv(path_farms)
df_farms = df_farms.iloc[:-2]

In [30]:
df_farms

,Name,OnshoreWind,OffShoreWind,PV,centroid_lon,centroid_lat
0,A,1150,0,150,-7.961187,54.781943
1,B,1330,0,470,-9.234611,53.783391
2,C,630,0,520,-7.960707,53.574123
3,D,380,0,260,-8.956443,52.905195
4,E,1830,390,560,-9.366938,52.168014
5,F,320,0,90,-9.106804,51.683789
6,G,370,370,490,-6.890747,53.892717
7,H1,630,0,450,-7.996979,52.564234
8,H2,860,1170,1260,-6.803512,52.670921
9,I,10,350,460,-8.354858,51.862738


In [31]:
df_capacity_solar = df_farms[['PV', 'centroid_lon', 'centroid_lat']]

In [32]:
df_capacity_solar = df_capacity_solar.rename({'PV':'MEC', 'centroid_lon':'x', 'centroid_lat':'y'}, axis='columns')
df_capacity_solar['capacity'] = 1.4*df_capacity_solar['MEC']

### Step 3: Run atlite

In [33]:
def get_cf_series_atlite(cutout_ie: atlite.Cutout, df_capacity: pd.DataFrame, panel_model: str, only_cf: bool = True):
    summed_time_series = np.zeros(cutout_ie.data['time'].shape)
    total_capacity_time_series  = np.zeros(cutout_ie.data['time'].shape)

    # Iterate over all capacity installations
    for idx, row in tqdm(df_capacity.reset_index().iterrows(), total= df_capacity.shape[0]):
        cap = row['capacity']
        mec = row['MEC']
        df_capacity_i = pd.DataFrame([row])
        layout = cutout_ie.layout_from_capacity_list(df_capacity_i, col="capacity")

        time_series = cutout_ie.pv(layout=layout,
                                    panel=atlite.convert.get_solarpanelconfig(panel_model),
                                    orientation=atlite.convert.get_orientation('latitude_optimal'),
                                    tracking=None)

        time_series = time_series.squeeze()
        time_series[time_series>mec] = mec

        capacity_time_series = time_series.copy()
        capacity_time_series[:] = mec

    # Add the mean CF time series to the total multiplied by the capacity (weight)
        summed_time_series += time_series[:]
        total_capacity_time_series += capacity_time_series[:]

    # Divide the total time series by the total IC to go back to CF
    cf_time_series = summed_time_series/total_capacity_time_series
    if only_cf:
        return cf_time_series
    return cf_time_series, summed_time_series, total_capacity_time_series

In [34]:
pv_cf, pv_gen, pv_ic = get_cf_series_atlite(cutout_ie=cutout_ie, df_capacity=df_capacity_solar, panel_model='KANENA', only_cf=False)

  0%|          | 0/12 [00:00<?, ?it/s]

[######                                  ] | 16% Completed | 6.36 s

/Users/aina/anaconda3/envs/atlite/lib/python3.11/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in log
  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed | 27.54 s


  8%|▊         | 1/12 [00:29<05:23, 29.44s/it]

...

 92%|█████████▏| 11/12 [04:42<00:26, 26.37s/it]

[######                                  ] | 16% Completed | 7.13 s

/Users/aina/anaconda3/envs/atlite/lib/python3.11/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in log
  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed | 27.75 s


100%|██████████| 12/12 [05:13<00:00, 26.10s/it]


In [35]:
pv_cf.to_netcdf('../data/pv_2030_cf_1940_2023.nc')